**Praktikum 2**

Generator Teks dengan RNN

---

Praktikum ini mendemonstrasikan cara melakukan genearsi text menggunakan RNN. Dataset yang digunkan adalah dataset Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Jika diberikan urutan karakter dari data ini ("Shakespear"), latih model untuk memprediksi karakter berikutnya dalam urutan ("e"). Urutan teks yang lebih panjang dapat dihasilkan dengan memanggil model berulang kali.

Note: Enable GPU acceleration to execute this notebook faster. In Colab: Runtime > Change runtime type > Hardware accelerator > GPU.

Tutorial ini menggunakan tf.keras dan eager execution. Berikut adalah contoh output ketika model dalam tutorial ini dilatih selama 30 epoch, dan dimulai dengan prompt "Q":

---

QUEENE:

I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:

Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:

The cause why then we are all resolved more sons.

VOLUMNIA:

O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:

But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:

I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m

---

Meskipun beberapa kalimat memiliki tata bahasa, sebagian besar tidak masuk akal. Model belum mempelajari arti kata-kata, namun anggap saja:

- Modelnya berbasis karakter. Saat pelatihan dimulai, model tidak mengetahui cara mengeja kata dalam bahasa Inggris, atau bahkan kata-kata tersebut merupakan satuan teks.
- Struktur keluarannya menyerupai sandiwara—blok teks umumnya dimulai dengan nama pembicara, dengan huruf kapital semua mirip dengan kumpulan data.
- Seperti yang ditunjukkan di bawah, model dilatih pada kumpulan teks kecil (masing-masing 100 karakter), dan masih mampu menghasilkan rangkaian teks yang lebih panjang dengan struktur yang koheren.

---

**Setup**

---

**Import TensorFlow**

In [1]:
import tensorflow as tf # digunakan untuk membangun dan melatih model machine learning
import numpy as np # digunakan untuk menangani array multidimensi dan operasi matematis
import os # digunakan untuk berinteraksi dengan sistem operasi
import time # digunakan untuk mengukur waktu

**Download Dataset Shakespeare**

Sesuaikan dengan lokasi data yang Anda punya.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt') # mengunduh file dari URL ke direktori cache Keras

**Load Data**

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8') # digunakan untuk membaca file teks yang dikodekan dalam UTF-8 dan mengembalikan konten file tersebut sebagai string
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters') # mencetak panjang dari string text

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250]) # Menampilkan 250 karakter pertama

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text)) # membuat vocab dengan set dari teks yang diurutkan
print(f'{len(vocab)} unique characters') # menampilkan panjang variabel vocab

65 unique characters


**Olah Teks**

**Vectorize Teks**

Sebelum training, Anda perlu mengonversi string menjadi representasi numerik. tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [6]:
example_texts = ['abcdefg', 'xyz'] # Membuat list example_texts
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8') # memecah teks yang diberikan menjadi urutan poin kode Unicode
chars # menampilkan variabel chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

sekarang buat tf.keras.layers.StringLookup layer:

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None) # membuat lapisan StringLookup TensorFlow yang dapat digunakan untuk mengkonversi token teks menjadi ID numerik

perintah diatas mengconvert token menjadi id

In [8]:
ids = ids_from_chars(chars) # mengkonversi urutan poin kode Unicode chars menjadi urutan ID numerik menggunakan lapisan StringLookup
ids # Menampilkan variabel ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Karena tujuan tutorial ini adalah untuk menghasilkan teks, penting juga untuk membalikkan representasi ini. Untuk ini Anda dapat menggunakan kode tf.keras.layers.StringLookup(..., invert=True).

Catatan: pada kode ini, daripada meneruskan kosakata asli yang dihasilkan dengan diurutkan(set(teks)) gunakan metode get_vocabulary() dari tf.keras.layers.StringLookup sehingga token [UNK] disetel dengan cara yang sama.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None) # membuat lapisan StringLookup TensorFlow dengan tambahan argument invert=True dan tidak menggunakan token mask

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor:

In [10]:
chars = chars_from_ids(ids) # mengkonversi urutan poin kode Unicode chars menjadi urutan ID numerik menggunakan lapisan StringLookup
chars # Menampilkan variabel ids

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Anda dapat menggunakan tf.strings.reduce_join untuk menggabungkan kembali karakter menjadi string.

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy() # menggabungkan nilai pada chars menjadi string

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1) # mengembalikan string yang digabungkan dari semua token teks dalam urutan ID numerik ids

**Prediksi**

---

Diberikan sebuah karakter, atau serangkaian karakter, karakter apa yang paling mungkin berikutnya? Ini adalah tugas yang harus Anda latih agar model dapat melakukannya. Masukan ke model akan berupa urutan karakter, dan Anda melatih model untuk memprediksi keluaran berupa karakter berikut pada setiap langkah waktu. Karena RNN mempertahankan keadaan internal yang bergantung pada elemen yang terlihat sebelumnya, mengingat semua karakter dihitung hingga saat ini, karakter apa selanjutnya?

**Membuat Trianing Set dan Target**

Selanjutnya bagilah teks menjadi contoh sequence. Setiap masukan sequence akan berisi karakter seq_length dari teks. Untuk setiap masukan sequence, target prediksi berisi teks dengan panjang yang sama, hanya digeser satu karakter ke kanan. Jadi, bagi teks menjadi beberapa bagian seq_length+1. Misalnya, seq_length adalah 4 dan teks kita adalah "Hello". Urutan masukannya adalah "Hell", dan urutan targetnya adalah "ello". Untuk melakukan ini, pertama-tama gunakan fungsi tf.data.Dataset.from_tensor_slices untuk mengonversi vektor teks menjadi aliran indeks karakter.

In [13]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8')) # mengkonversi semua karakter dalam teks text menjadi ID numerik menggunakan lapisan StringLookup ids_from_chars
all_ids # Menamplikan variabel all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [14]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # membuat dataset TensorFlow dari urutan ID numerik all_ids

In [15]:
for ids in ids_dataset.take(10): # Melakukan perulangan pada 10 elemen pertama dari dataset ids_dataset
    print(chars_from_ids(ids).numpy().decode('utf-8')) #  mencetak token teks yang sesuai dengan ID numerik ids

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100 # Inisialisasi variabel seq_length

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True) # membuat batch tensor ID numerik dari dataset ids_dataset, dengan ID urutan selanjutnya sebagai target dan mengabaikan batch terakhir jika panjangnya kurang dari seq_length+1

for seq in sequences.take(1): # Melakikan perulangan nilai pertama dari data sequence
  print(chars_from_ids(seq)) # mencetak token teks yang sesuai dengan urutan ID numerik seq

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string:

In [18]:
for seq in sequences.take(5): # Melakukan perulangan pada 5 elemen pertama dari dataset ids_dataset
    print(text_from_ids(seq).numpy()) # mencetak teks yang sesuai dengan urutan ID numerik seq

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


Untuk pelatihan, Anda memerlukan kumpulan data pasangan (input, label). Dimana input dan label merupakan urutan. Pada setiap langkah waktu, inputnya adalah karakter saat ini dan labelnya adalah karakter berikutnya. Berikut adalah fungsi yang mengambil urutan sebagai masukan, menduplikasi, dan menggesernya untuk menyelaraskan masukan dan label untuk setiap langkah waktu:

In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1] # mengambil seluruh nilai kecuali nilai terakhir
    target_text = sequence[1:] # mengambil seluruh nilai kecuali nilai pertama
    return input_text, target_text # mengembalikan variabel input_text dan target_text

In [20]:
split_input_target(list("Tensorflow")) # menjalankan function split_input_target dengan argument list 'Tensorflow'

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target) # memetakan setiap elemen dalam dataset sequences ke urutan ID numerik yang berisi input teks dan target teks

In [22]:
for input_example, target_example in dataset.take(1): # melakukan perulangan pada elemen pertama dataset dan membaginya ke variabel input_example dan target_example
    print("Input :", text_from_ids(input_example).numpy()) # menampilkan nilai input_example
    print("Target:", text_from_ids(target_example).numpy()) # menampilkan nilai target_example

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


**Membuat Batch Training**

Anda menggunakan tf.data untuk membagi teks menjadi sequence yang dapat diatur. Namun sebelum memasukkan data ini ke dalam model, Anda perlu mengacak data dan mengemasnya ke dalam batch.

In [23]:
# Batch size
BATCH_SIZE = 64 # Inisialisasi variabel BATCH_SIZE

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000 # Inisialisasi variabel BUFFER_SIZE

dataset = (
    dataset # Membuat dataset
    .shuffle(BUFFER_SIZE) # mengacak urutan elemen dalam dataset dan menyimpan elemen-elemen yang diacak berdasarkan buffer_size
    .batch(BATCH_SIZE, drop_remainder=True) # mengelompokkan elemen-elemen dalam dataset menjadi batch dan  elemen-elemen yang tidak dapat dibagi oleh BATCH_SIZE akan dibuang
    .prefetch(tf.data.experimental.AUTOTUNE)) # TensorFlow secara otomatis menentukan ukuran buffer yang optimal untuk prefetch

dataset # menampilkan variabel dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

**Buat Model**

---

Bagian ini mendefinisikan model sebagai subkelas keras.Model (untuk lebih detilnya, lihat [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)).

Model yang kita bangun memiliki 3 lapisan neural network :

- tf.keras.layers.Embedding: Lapisan masukan. Tabel pencarian yang dapat dilatih yang akan memetakan setiap karakter-ID ke vektor dengan dimensi embedding_dim;
- tf.keras.layers.GRU: lapisan RNN dengan ukuran unit=rnn_units (Anda juga dapat menggunakan lapisan LSTM di sini.)
- tf.keras.layers.Dense: Lapisan keluaran, dengan keluaran vocab_size. Ini menghasilkan satu logit untuk setiap karakter dalam kosakata. Ini adalah log kemungkinan setiap karakter menurut model.

In [24]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary()) # menghitung ukuran vocabulary dari lapisan StringLookup ids_from_chars

# The embedding dimension
embedding_dim = 256 # Inisialisasi variabel embedding_dim

# Number of RNN units
rnn_units = 1024 # Inisialisasi variabel rnn_units

In [25]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # membuat embedding layer untuk memproses urutan ID numerik yang dihasilkan oleh lapisan StringLookup
    self.gru = tf.keras.layers.GRU(rnn_units, # Jumlah unit sebanyak rnn_units
                                   return_sequences=True, # mengembalikan urutan vektor numerik yang sama dengan panjang urutan input
                                   return_state=True) # membuat lapisan GRU untuk memproses urutan vektor numerik yang dihasilkan oleh lapisan embedding dan mengembalikan keadaan internal dari lapisan GRU
    self.dense = tf.keras.layers.Dense(vocab_size) # membuat layer dense dengan argument vacab_size

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs # menyimpan variabel inputs ke dalam variabel x
    x = self.embedding(x, training=training) # memproses urutan ID numerik x menggunakan lapisan embedding self.embedding dengan set mode training berdasarkan variabel training
    if states is None:
      states = self.gru.get_initial_state(x) # mendapatkan keadaan awal dari lapisan GRU
    x, states = self.gru(x, initial_state=states, training=training) # memproses urutan vektor numerik x menggunakan lapisan GRU self.gru dengan keadaan awal states dan mode berdasarkan variabel training
    x = self.dense(x, training=training) # memproses urutan vektor numerik x menggunakan lapisan dense

    if return_state:
      return x, states # mengembalikan nilai x dan states
    else:
      return x # hanya mengembalikan nilai x

In [26]:
model = MyModel( # Memanggil function MyModel()
    vocab_size=vocab_size, # menambahkan argument berupa variabel vocab_size
    embedding_dim=embedding_dim, # menambahkan argument berupa variabel embedding_dim
    rnn_units=rnn_units) # menambahkan argument berupa variabel rnn_units

Untuk setiap karakter, model mencari penyematan, menjalankan GRU satu langkah waktu dengan penyematan sebagai masukan, dan menerapkan dense layer untuk menghasilkan log yang memprediksi kemungkinan log karakter berikutnya:

![image](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_training.png?raw=1)

Note: Untuk pelatihan Anda bisa menggunakan model keras.Sequential di sini. Untuk menghasilkan teks nanti, Anda harus mengelola status internal RNN. Akan lebih mudah untuk memasukkan opsi input dan output status di awal, daripada mengatur ulang arsitektur model nanti. untuk detailnya bisa dilihat Keras RNN guide.

---

**Uji Model**

Coba jalankan model dan cek apakah sidah sesuai dengan output

pertama, cek bentuk dari output

In [27]:
for input_example_batch, target_example_batch in dataset.take(1): # Melukan perulangan dengan elemen pertama yang dibagi dalam variabel input_example_batch dan target_example_batch
    example_batch_predictions = model(input_example_batch) # menghasilkan prediksi untuk batch input input_example_batch menggunakan model
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)") # menampilkan ukuran example_batch_predictions

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


Dalam contoh di atas, panjang urutan masukan adalah 100 tetapi model dapat dijalankan pada masukan dengan panjang berapa pun:

In [28]:
model.summary() # mencetak ringkasan dari model machine learning yang telah dibuat

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Untuk mendapatkan prediksi aktual dari model, Anda perlu mengambil sampel dari distribusi keluaran, untuk mendapatkan indeks karakter aktual. Distribusi ini ditentukan oleh logit pada kosakata karakter.

Catatan: Penting untuk mengambil sampel dari distribusi ini karena mengambil argmax dari distribusi tersebut dapat dengan mudah membuat model terjebak dalam infinote loop. Cobalah untuk contoh pertama di batch:

In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) # mengambil sampel dari distribusi kategorikal yang ditentukan oleh vektor probabilitas example_batch_predictions[0] dengan jumlah sampel yang diambil adalah 1
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy() # menghapus dimensi dari tensor dan mengubah tensor sampled_indices menjadi array NumPy

Hal ini memberi kita, pada setiap langkah waktu, prediksi indeks karakter berikutnya:

In [30]:
sampled_indices # menampilkan array variabel sampled_indices

array([48, 23, 41, 56, 30, 51, 64, 16, 26, 65, 17, 16, 31,  8, 37, 51, 19,
        6, 35,  8, 47, 47, 46, 44, 17, 14, 15, 57, 46, 55, 38, 40, 58, 47,
        4, 26, 47, 21, 38, 29, 61, 19, 25, 16, 65, 24, 63, 52, 33, 20,  6,
       55, 55, 62, 38,  8,  3, 29, 39, 31,  7, 16, 18, 13, 57, 63, 62, 62,
       47, 50, 20, 57, 36, 53, 34, 38,  0, 10,  8, 61, 25, 50, 12, 10, 55,
       31, 25, 19, 18,  5, 45, 55, 45, 37, 52, 33, 64,  7, 31, 25],
      dtype=int64)

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [31]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy()) # mencetak input pertama dalam batch input_example_batch
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy()) # mencetak prediksi karakter berikutnya untuk input pertama dalam batch input_example_batch

Input:
 b"all your tears! I am your sorrow's nurse,\nAnd I will pamper it with lamentations.\n\nDORSET:\nComfort, "

Next Char Predictions:
 b"iJbqQlyCMzDCR-XlF'V-hhgeDABrgpYash$MhHYPvFLCzKxmTG'ppwY-!PZR,CE?rxwwhkGrWnUY[UNK]3-vLk;3pRLFE&fpfXmTy,RL"


**Train Model**

Pada titik ini permasalahan dapat dianggap sebagai permasalahan klasifikasi standar. Permasalahan dapat disimpulkan dengan : Berdasarkan status RNN sebelumnya, dan masukan langkah kali ini, prediksi kelas karakter berikutnya.

**Tambahan optimizer dan fungsi loss**

---

loss function tf.keras.losses.sparse_categorical_crossentropy standar berfungsi dalam kasus ini karena diterapkan di seluruh dimensi terakhir prediksi. Karena model Anda mengembalikan logits, Anda perlu mengatur flag from_logits.

In [32]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True) # membuat fungsi kerugian Sparse Categorical Crossentropy dengan menerima logits sebagai input

In [33]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions) # menghitung rata-rata kerugian untuk batch input example_batch_predictions
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)") # menampilkan ukuran example_batch_predictions
print("Mean loss:        ", example_batch_mean_loss) # menampilkan mean loss

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.188815, shape=(), dtype=float32)


Model yang baru diinisialisasi tidak boleh terlalu yakin dengan dirinya sendiri, semua log keluaran harus memiliki besaran yang sama. Untuk mengonfirmasi hal ini, Anda dapat memeriksa bahwa eksponensial dari loss rata-rata harus kira-kira sama dengan ukuran kosakata. Loss yang jauh lebih tinggi berarti model tersebut yakin akan jawaban yang salah, dan memiliki inisialisasi yang buruk:

In [34]:
tf.exp(example_batch_mean_loss).numpy() # menghitung eksponen dari rata-rata kerugian untuk batch input

65.94461

Konfigurasikan prosedur pelatihan menggunakan metode tf.keras.Model.compile. Gunakan tf.keras.optimizers.Adam dengan argumen default dan fungsi loss.

In [35]:
model.compile(optimizer='adam', loss=loss) # Compile model denga optimizer ADAM dan argument variabel loss

**Konfigurasi Checkpoints**

Gunakan tf.keras.callbacks.ModelCheckpoint untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [36]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints' # membuat direktori untuk menyimpan checkpoint model
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}") # membuat prefix nama file untuk checkpoint model

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True) # menyimpan checkpoint model machine learning ke direktori checkpoint_dir dengan prefix nama file ckpt_{epoch} dan yang disimpan hanya berupa bobotnya

**Lakukan Proses Training**

Agar waktu pelatihan tidak terlalu lama, gunakan 10 epoch untuk melatih model. Di Colab, setel runtime ke GPU untuk pelatihan yang lebih cepat.

In [37]:
EPOCHS = 20 # Inisialisasi variabel EPOCHS

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback]) # melatih model  dengan dataset dataset selama sebanyak EPOCHS, dengan menggunakan callback untuk menyimpan checkpoint model

Epoch 1/20


172/172 [==============================] - 1074s 6s/step - loss: 2.7315
Epoch 2/20
 38/172 [=====>........................] - ETA: 12:59 - loss: 2.1421

**Generate Teks**

---

Cara termudah untuk menghasilkan teks dengan model ini adalah dengan menjalankannya dalam loop, dan menyimpan status internal model saat Anda menjalankannya.

![image](https://github.com/tensorflow/text/blob/master/docs/tutorials/images/text_generation_sampling.png?raw=1)

Setiap kali Anda memanggil model, Anda memasukkan beberapa teks dan state internal. Model mengembalikan prediksi untuk karakter berikutnya dan state barunya. Masukkan kembali prediksi dan state ke model untuk terus menghasilkan teks.

---

Berikut ini membuat prediksi satu langkah:

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None] # membuat tensor yang berisi ID token [UNK]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())]) # membuat tensor sparse yang berisi nilai -float('inf') untuk setiap indeks token [UNK]
    self.prediction_mask = tf.sparse.to_dense(sparse_mask) # mengkonversi tensor sparse sparse_mask menjadi tensor padat

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8') # memecah string yang terkandung dalam tensor inputs menjadi karakter-karakter Unicode menggunakan encoding UTF-8
    input_ids = self.ids_from_chars(input_chars).to_tensor() # mengkonversi input karakter input_chars menjadi tensor ID token

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True) # Menghitung prediksi model machine learning self.model untuk input input_ids dan mengembalikan prediksi dan status model.
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :] # mengambil prediksi model untuk karakter berikutnya dari input input_ids
    predicted_logits = predicted_logits/self.temperature # meningkatkan keragaman dan kreativitas dari prediksi model
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask # mengabaikan token [UNK] saat memprediksi karakter berikutnya

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1) # mengambil satu sampel karakter berikutnya dari prediksi model
    predicted_ids = tf.squeeze(predicted_ids, axis=-1) #  menghapus dimensi tambahan dari tensor predicted_ids

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids) #  mengkonversi ID karakter yang diprediksi menjadi karakter yang sebenarnya

    # Return the characters and model state.
    return predicted_chars, states # mengembalikan prediksi karakter dan statets

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars) # membuat model one-step untuk menghasilkan teks

Jalankan secara berulang untuk menghasilkan beberapa teks. Melihat teks yang dihasilkan, Anda akan melihat model mengetahui kapan harus menggunakan huruf besar, membuat paragraf, dan meniru kosakata menulis seperti Shakespeare. Karena sedikitnya jumlah epoch pelatihan, model belum belajar membentuk kalimat runtut.

In [ ]:
start = time.time() # mencatat waktu mulai dari suatu proses
states = None # Memberikan nilai awal None pada states
next_char = tf.constant(['ROMEO:']) # membuat tensor konstan yang berisi karakter ROMEO, yang nilainya tidak dapat diubah
result = [next_char] # membuat list yang berisi tensor konstan

for n in range(1000): # Melakukan 1000 perulangan
  next_char, states = one_step_model.generate_one_step(next_char, states=states) # menghasilkan karakter berikutnya dari input teks menggunakan model one-step one_step_model
  result.append(next_char) # Menambahkan nilai next_char ke list result

result = tf.strings.join(result) # menggabungkan semua elemen dari list result menjadi satu string.
end = time.time() # mencatat waktu proses berakhir
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) # mencetak teks yang dihasilkan oleh model one-step, diikuti oleh garis bawah sepanjang 80 karakter dalam format decode utf-8
print('\nRun time:', end - start) # Menampilkan waktu runtime dari selisih waktu mulai dan selesai

ROMEO:
Thou dost advise me: O, thank you'lo know how wears,
Which were in, to quicken you; it is no other.

KING HENRY PI:
We master, sund it that, and thankful voices, sir; for thou
art seen such sister with you. We pray, we must speak well.

SAMPSON:
Not yet. in the devil is my love's perduced!

Second Messenger:
My liege, that I may saw her here, yet doth,
Pursued my men make the accusation do I swear
To meet you like a thorny welt company:
And this same headly let give no other things.

WARWICK:
Son George Shall the seat to this care?

Servant:
God gentle honest gentlemen! im he thereby
These pride he look'd upon our hands
This trensure he was that tiged in my tent
Vidow in the falation. We march on, Without I
As better betiment in a subject length.
3 KING HENRY VI

QUEEN MARGARET:
Thou liest, Citizen:
Neighbours, Glumio maked Marianatise,
Which never frown'd in blood and banish'd Hereford,
To frank up in your tidings. Gloumstand you
Of my poor hold, how for your news?--and cries '

Hal termudah yang dapat Anda lakukan untuk meningkatkan hasil adalah dengan melatihnya lebih lama (coba EPOCHS = 30). Anda juga dapat bereksperimen dengan string awal yang berbeda, mencoba menambahkan lapisan RNN lain untuk meningkatkan akurasi model, atau menyesuaikan parameter suhu untuk menghasilkan prediksi yang kurang lebih acak.

Jika Anda ingin model menghasilkan teks lebih cepat, hal termudah yang dapat Anda lakukan adalah membuat teks secara batch. Pada contoh di bawah, model menghasilkan 5 keluaran dalam waktu yang hampir sama dengan waktu yang dibutuhkan untuk menghasilkan 1 keluaran di atas.

In [ ]:
start = time.time() # mencatat waktu mulai dari suatu proses
states = None # Memberikan nilai awal None pada states
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:']) # membuat tensor konstan yang berisi lima karakter ROMEO, yang nilainya tidak dapat diubah
result = [next_char] # membuat list yang berisi tensor konstan

for n in range(1000): # Melakukan 1000 perulangan
  next_char, states = one_step_model.generate_one_step(next_char, states=states) # menghasilkan karakter berikutnya dari input teks menggunakan model one-step one_step_model
  result.append(next_char) # Menambahkan nilai next_char ke list result

result = tf.strings.join(result) # menggabungkan semua elemen dari list result menjadi satu string.
end = time.time() # mencatat waktu proses berakhir
print(result, '\n\n' + '_'*80) # mencetak teks yang dihasilkan oleh model one-step, diikuti oleh garis bawah sepanjang 80 karakter
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80) # mencetak teks yang dihasilkan oleh model one-step, diikuti oleh garis bawah sepanjang 80 karakter  dalam format decode utf-8
print('\nRun time:', end - start) # Menampilkan waktu runtime dari selisih waktu mulai dan selesai

tf.Tensor(
[b"ROMEO:\nLady's heart let this sad keeping him.\n\nNORTHUMBERLAND:\nTrue bawner Edward be an almost circues,\nWhen he dared sir, we have not in this fellow of aged\nTo sigh me this sea framed, must give thee thin:\nThrow dwelling in the sun season: therefore, great\nThe stars, which that can I condem these death,\nYour girth shoes wounds old.\n\nDUKE VINCENTIO:\nWill't please you, sir?\n\nFLOReLLA:\nIt is appear: may steal upon the crown,\nTo strong a tale flower. Now says her blows.\n\nKING Edward, I will well I know my departure\nTo some hold man. Only wife, and see a troop:\nUnless, by heaven, I clear the Lady friends,\nAnd, for the fault length pretty one, so much help to two\nGalled.\n\nFirst Lord:\nWhat for I have, thou dost,\nAnd she these banners only to our reports.\nAy, and therefore, no done! Cath's but needs but in a cold cold fire:\nNow then we are unparallel'd.\n\nPRINCE:\nAnd since I cannot, good, forbear atide,\nI may not leave this sapsies behowe the offic

**Ekspor Model Generator**

Model satu langkah ini dapat dengan mudah disimpan dan digunakan kembali, memungkinkan Anda menggunakannya di mana pun tf.saved_model diterima.

In [ ]:
tf.saved_model.save(one_step_model, 'one_step') # menyimpan model one-step one_step_model ke direktori one_step
one_step_reloaded = tf.saved_model.load('one_step') # memuat model one-step yang disimpan di direktori one_step ke dalam variabel one_step_reloaded

In [ ]:
states = None # Memberikan nilai awal None pada states
next_char = tf.constant(['ROMEO:']) # membuat tensor konstan yang berisi karakter ROMEO, yang nilainya tidak dapat diubah
result = [next_char] # membuat list yang berisi tensor konstan

for n in range(100): # Melakukan 100 perulangan
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states) # menghasilkan karakter berikutnya dari input teks menggunakan model one-step one_step_model
  result.append(next_char) # Menambahkan nilai next_char ke list result

print(tf.strings.join(result)[0].numpy().decode("utf-8")) # # menggabungkan semua elemen dari list result menjadi satu string dan menampilkan hasilnya dalam format decode utf-8

ROMEO:
The senators of all ares unharry: spur
detice, your scorn'd, take up some other in our will,
Which 
